In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
!ls /content/drive/MyDrive/PML_project/dataset

X_test_1d.npy			y_grain_cond_ka_test.npy
X_train_1d.npy			y_grain_cond_ka_train.npy
X_val_1d.npy			y_grain_cond_ka_val.npy
y_grain_cond_filt_ka_test.npy	y_raw_ka_test.npy
y_grain_cond_filt_ka_train.npy	y_raw_ka_train.npy
y_grain_cond_filt_ka_val.npy	y_raw_ka_val.npy


In [3]:
path = 'dataset/'
X_train = np.load(path+'X_train_1d.npy')
X_test = np.load(path+'X_test_1d.npy')
X_val = np.load(path+'X_val_1d.npy') #this may not be required for now

y_train = np.load(path+'y_grain_cond_ka_train.npy')
y_train = np.where(y_train>=0, 1, 0)
y_test = np.load(path+'y_grain_cond_ka_test.npy')
y_test = np.where(y_test>=0, 1, 0)
y_val = np.load(path+'y_grain_cond_ka_val.npy') #this may not be required for now
y_val = np.where(y_val>=0, 1, 0)

In [4]:
print(X_train.shape, X_test.shape, X_val.shape)
print(y_train.shape, y_test.shape, y_val.shape)

(3185, 4) (970, 4) (676, 4)
(3185,) (970,) (676,)


In [5]:
del_idx = np.unique(np.where(np.isnan(X_train))[0])
del_idx_test = np.unique(np.where(np.isnan(X_test))[0])
X_train = np.delete(X_train, del_idx, axis=0)
X_test = np.delete(X_test, del_idx_test, axis=0)

In [6]:
y_train = np.delete(y_train, del_idx, axis=0)
y_test = np.delete(y_test, del_idx_test, axis=0)

In [7]:
print(X_train.shape, X_test.shape, X_val.shape)
print(y_train.shape, y_test.shape, y_val.shape)

(2074, 4) (632, 4) (676, 4)
(2074,) (632,) (676,)


In [10]:
import os
from absl import app
from absl import flags
from matplotlib import cm
from matplotlib import figure
from matplotlib.backends import backend_agg
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

tf.random.set_seed(10)

tfd = tfp.distributions
flags.DEFINE_string('f', '', 'kernel')

flags.DEFINE_float('learning_rate',
                   default=0.01,
                   help='Initial learning rate.')
flags.DEFINE_integer('num_epochs',
                     default=100,
                     help='Number of epochs to run.')
flags.DEFINE_integer('batch_size',
                     default=128,
                     help='Batch size.')
flags.DEFINE_string(
    'model_dir',
    default=os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                         'logistic_regression/'),
    help="Directory to put the model's fit.")
flags.DEFINE_integer('num_examples',
                     default=2074,
                     help='Number of datapoints to generate.')
flags.DEFINE_integer('num_monte_carlo',
                     default=50,
                     help='Monte Carlo samples to visualize weight posterior.')

FLAGS = flags.FLAGS
# The dimensions of the example data, ie shape=(256, 2)
NUM_DIMENSIONS = 4

MODEL = None

####Delete all flags before declare#####

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)



def visualize_decision(features, labels, true_w_b, candidate_w_bs, fname):
  """Utility method to visualize decision boundaries in R^2.
  Args:
    features: Input points, as a Numpy `array` of shape `[num_examples, 2]`.
    labels: Numpy `float`-like array of shape `[num_examples, 1]` giving a
    label for each point.
    true_w_b: A `tuple` `(w, b)` where `w` is a Numpy array of
    shape `[2]` and `b` is a scalar `float`, interpreted as a
    decision rule of the form `dot(features, w) + b > 0`.
    candidate_w_bs: Python `iterable` containing tuples of the same form as
    true_w_b.
    fname: The filename to save the plot as a PNG image (Python `str`).
  """
  fig = figure.Figure(figsize=(6, 6))
  canvas = backend_agg.FigureCanvasAgg(fig)
  ax = fig.add_subplot(1, 1, 1)
  ax.scatter(features[:, 0], features[:, 1],
             c=np.float32(labels[:, 0]),
             cmap=cm.get_cmap('binary'),
             edgecolors='k')

  def plot_weights(w, b, **kwargs):
    w1, w2, w3, w4 = w
    x1s = np.linspace(-1, 1, 100)
    x2s = -(w1  * x1s + b) / w2
    ax.plot(x1s, x2s, **kwargs)

  for w, b in candidate_w_bs:
    plot_weights(w, b,
                 alpha=1./np.sqrt(len(candidate_w_bs)),
                 lw=1, color='blue')

  if true_w_b is not None:
    plot_weights(*true_w_b, lw=4,
                 color='green', label='true separator')

  ax.set_xlim([-1.5, 1.5])
  ax.set_ylim([-1.5, 1.5])
  ax.legend()

  canvas.print_figure(fname, format='png')
  print('saved {}'.format(fname))


def toy_logistic_data(num_examples, input_size=2, weights_prior_stddev=5.0):
  """Generates synthetic data for binary classification.
  Args:
    num_examples: The number of samples to generate (scalar Python `int`).
    input_size: The input space dimension (scalar Python `int`).
    weights_prior_stddev: The prior standard deviation of the weight
        vector. (scalar Python `float`).
  Returns:
    random_weights: Sampled weights as a Numpy `array` of shape
        `[input_size]`.
    random_bias: Sampled bias as a scalar Python `float`.
    design_matrix: Points sampled uniformly from the cube `[-1,
        1]^{input_size}`, as a Numpy `array` of shape `(num_examples,
        input_size)`.
    labels: Labels sampled from the logistic model `p(label=1) =
        logistic(dot(features, random_weights) + random_bias)`, as a Numpy
        `int32` `array` of shape `(num_examples, 1)`.
  """
  random_weights = weights_prior_stddev * np.random.randn(input_size)
  random_bias = np.random.randn()
  design_matrix = np.random.rand(num_examples, input_size) * 2 - 1
  logits = np.reshape(
      np.dot(design_matrix, random_weights) + random_bias,
      (-1, 1))
  p_labels = 1. / (1 + np.exp(-logits))
  labels = np.int32(p_labels > np.random.rand(num_examples, 1))
  return random_weights, random_bias, np.float32(design_matrix), labels

def real_logistic_data(num_examples, input_size=4):
  random_weights = np.random.randn(input_size)
  random_bias = np.random.randn()
  design_matrix = X_train
  labels = y_train.reshape((-1,1))
  return random_weights, random_bias, np.float32(design_matrix), labels

class ToyDataSequence(tf.keras.utils.Sequence):
  """Creates a sequence of labeled points from provided numpy arrays."""

  def __init__(self, features, labels, batch_size):
    """Initializes the sequence.
    Args:
      features: Numpy `array` of features, indexed by the first dimension.
      labels: Numpy `array` of features, with the same first dimension as
              `features`.
      batch_size: Integer, number of elements in each training batch.
    """
    self.features = features
    self.labels = labels
    self.batch_size = batch_size

  def __len__(self):
    return int(np.ceil(len(self.features) / self.batch_size))

  def __getitem__(self, idx):
    batch_x = self.features[self.batch_size * idx : self.batch_size * (idx + 1)]
    batch_y = self.labels[self.batch_size * idx: self.batch_size * (idx + 1)]
    return batch_x, batch_y


def create_model(num_samples, num_dimensions):
  """Creates a Keras model for logistic regression.
  Args:
   num_samples: Integer for number of training samples.
   num_dimensions: Integer for number of features in dataset.
  Returns:
    model: Compiled Keras model.
  """
  # KL divergence weighted by the number of training samples, using
  # lambda function to pass as input to the kernel_divergence_fn on
  # flipout layers.
  kl_divergence_function = (lambda q, p, _: tfd.kl_divergence(q, p) /  # pylint: disable=g-long-lambda
                            tf.cast(num_samples, dtype=tf.float32))

  # Define a logistic regression model as a Bernoulli distribution
  # parameterized by logits from a single linear layer. We use the Flipout
  # Monte Carlo estimator for the layer: this enables lower variance
  # stochastic gradients than naive reparameterization.
  input_layer = tf.keras.layers.Input(shape=num_dimensions)
  dense_layer = tfp.layers.DenseFlipout(
      units=1,
      activation='sigmoid',
      kernel_posterior_fn=tfp.layers.default_mean_field_normal_fn(),
      bias_posterior_fn=tfp.layers.default_mean_field_normal_fn(),
      kernel_divergence_fn=kl_divergence_function)(input_layer)

  # Model compilation.
  model = tf.keras.Model(inputs=input_layer, outputs=dense_layer)
  optimizer = tf.keras.optimizers.Adam(lr=FLAGS.learning_rate)
  # We use the binary_crossentropy loss since this toy example contains
  # two labels. The Keras API will then automatically add the
  # Kullback-Leibler divergence (contained on the individual layers of
  # the model), to the cross entropy loss, effectively
  # calcuating the (negated) Evidence Lower Bound Loss (ELBO)
  model.compile(optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
  return model


def main(argv):
  #del_all_flags(tf.flags.FLAGS)
  del argv
  if tf.io.gfile.exists(FLAGS.model_dir):
    print('Warning: deleting old log directory at {}'.format(FLAGS.model_dir))
    tf.io.gfile.rmtree(FLAGS.model_dir)
  tf.io.gfile.makedirs(FLAGS.model_dir)

  # Generate a toy classification dataset.
  w_true, b_true, features, labels = real_logistic_data(FLAGS.num_examples)
  # w_true, b_true, features, labels = toy_logistic_data(FLAGS.num_examples)
  toy_logistic_sequence = ToyDataSequence(features, labels, FLAGS.batch_size)

  # Define and train a bayesian logistic regression model.
  model = create_model(FLAGS.num_examples, NUM_DIMENSIONS)
  model.save('drive/MyDrive/PML_project/blr')
  model.fit(toy_logistic_sequence, epochs=FLAGS.num_epochs,
            shuffle=True)
  print(model.summary())
  MODEL = model
  # return model
  # Visualize some draws from the weights posterior.
  # candidate_w_bs = [(model.layers[-1].kernel_posterior.sample().numpy(),
  #                    model.layers[-1].bias_posterior.sample().numpy())
  #                   for _ in range(FLAGS.num_monte_carlo)]
  # visualize_decision(features, labels, (w_true, b_true),
  #                    candidate_w_bs,
  #                    fname=os.path.join(FLAGS.model_dir,
  #                                       'weights_inferred.png'))


if __name__ == '__main__':
  app.run(main)

/usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/layers/util.py:102: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=trainable)
/usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/layers/util.py:112: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=trainable)
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


INFO:tensorflow:Assets written to: drive/MyDrive/PML_project/blr/assets


I0319 04:16:44.157793 140027568953216 builder_impl.py:780] Assets written to: drive/MyDrive/PML_project/blr/assets


Epoch 1/100
17/17 [==============================] - 4s 4ms/step - loss: 0.6960 - binary_accuracy: 0.5497
Epoch 2/100
17/17 [==============================] - 0s 3ms/step - loss: 0.6826 - binary_accuracy: 0.6133
Epoch 3/100
17/17 [==============================] - 0s 4ms/step - loss: 0.6772 - binary_accuracy: 0.5849
Epoch 4/100
17/17 [==============================] - 0s 5ms/step - loss: 0.6714 - binary_accuracy: 0.5873
Epoch 5/100
17/17 [==============================] - 0s 5ms/step - loss: 0.6784 - binary_accuracy: 0.5853
Epoch 6/100
17/17 [==============================] - 0s 5ms/step - loss: 0.6748 - binary_accuracy: 0.5824
Epoch 7/100
17/17 [==============================] - 0s 4ms/step - loss: 0.6736 - binary_accuracy: 0.5950
Epoch 8/100
17/17 [==============================] - 0s 4ms/step - loss: 0.6764 - binary_accuracy: 0.5969
Epoch 9/100
17/17 [==============================] - 0s 4ms/step - loss: 0.6739 - binary_accuracy: 0.6061
Epoch 10/100
17/17 [==========================

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
model = tf.keras.models.load_model('drive/MyDrive/PML_project/blr')

In [14]:
y_pred = model.predict(X_test)
y_pred_bin = np.where(y_pred>=0.5,1,0)
def accuracy(y_true, y_pred):
  count = 0
  for i in range(len(y_true)):
    if y_true[i] == y_pred[i]:
      count += 1
  return count/len(y_true)

In [19]:
accuracy(y_test, y_pred_bin.ravel())

0.6218354430379747